# Day 21 

In [ ]:
from collections import deque

# Define the keypads
numeric_keypad = {
    '7': (0, 0), '8': (0, 1), '9': (0, 2),
    '4': (1, 0), '5': (1, 1), '6': (1, 2),
    '1': (2, 0), '2': (2, 1), '3': (2, 2),
    '0': (3, 1), 'A': (3, 2)
}

directional_keypad = {
    '^': (0, 1), '<': (1, 0), 'v': (1, 1), '>': (1, 2), 'A': (0, 2)
}

# BFS to compute shortest path on a keypad
def bfs(start, goal, keypad):
    reverse_keypad = {v: k for k, v in keypad.items()}
    start_pos = keypad[start]
    goal_pos = keypad[goal]

    # Valid moves: up, down, left, right
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    queue = deque([(start_pos, "")])
    visited = set([start_pos])
    
    while queue:
        current, path = queue.popleft()
        
        if current == goal_pos:
            return path + "A"  # Add A to press the button
        
        for dx, dy in moves:
            new_pos = (current[0] + dx, current[1] + dy)
            if new_pos in reverse_keypad and new_pos not in visited:
                visited.add(new_pos)
                direction = reverse_keypad[new_pos]
                queue.append((new_pos, path + direction))

# Recursive function to find the sequence for each code
def find_sequence(code, numeric_keypad, directional_keypad):
    sequence = ""
    current_position = 'A'  # Initial position on the numeric keypad
    
    for char in code:
        # Find the sequence for the current character
        sub_sequence = bfs(current_position, char, numeric_keypad)
        sequence += sub_sequence
        # Reset to the next keypad's initial position
        current_position = char
    
    return sequence

# Calculate complexities
def calculate_complexities(codes):
    total_complexity = 0
    
    for code in codes:
        sequence = find_sequence(code, numeric_keypad, directional_keypad)
        length = len(sequence)
        numeric_part = int(code.lstrip('0').rstrip('A'))
        total_complexity += length * numeric_part
    
    return total_complexity

# Input codes

#540A
#582A
#169A
#593A
#579A

codes = ["540A", "582A", "169A", "593A", "579A"]

# Compute the total complexity
result = calculate_complexities(codes)
print("Total Complexity:", result)


In [ ]:
from collections import deque

def get_numeric_keypad():
    return {
        (0, 0): '7', (0, 1): '8', (0, 2): '9',
        (1, 0): '4', (1, 1): '5', (1, 2): '6',
        (2, 0): '1', (2, 1): '2', (2, 2): '3',
        (3, 1): '0', (3, 2): 'A'
    }

def get_directional_keypad():
    return {
        (0, 1): '^', (0, 2): 'A',
        (1, 0): '<', (1, 1): 'v', (1, 2): '>'
    }

def get_numeric_coords():
    return {v: k for k, v in get_numeric_keypad().items()}

def get_directional_coords():
    return {v: k for k, v in get_directional_keypad().items()}

def get_neighbors(keypad, current_pos):
    neighbors = []
    row, col = current_pos
    for dr, dc, move in [(0, 1, '>'), (0, -1, '<'), (1, 0, 'v'), (-1, 0, '^')]:
        new_row, new_col = row + dr, col + dc
        if (new_row, new_col) in keypad:
            neighbors.append(((new_row, new_col), move))
    return neighbors

def solve_keypad_puzzle(codes):
    total_complexity = 0

    numeric_keypad = get_numeric_keypad()
    numeric_coords = get_numeric_coords()
    dir_keypad = get_directional_keypad()
    dir_coords = get_directional_coords()

    def find_shortest_sequence(start_pos, target_sequence, keypad, get_coords):
        queue = deque([(start_pos, "")])
        visited = {start_pos}
        coords_map = get_coords()

        while queue:
            current_pos, current_moves = queue.popleft()

            if not target_sequence:
                return current_moves

            # Try pressing 'A'
            if keypad.get(current_pos) == target_sequence[0]:
                remaining_moves = find_shortest_sequence(current_pos, target_sequence[1:], keypad, get_coords)
                if remaining_moves is not None:
                    return current_moves + 'A' + remaining_moves

            # Try moving
            for neighbor_pos, move in get_neighbors(keypad, current_pos):
                if neighbor_pos not in visited:
                    visited.add(neighbor_pos)
                    queue.append((neighbor_pos, current_moves + move))
        return None

    def find_shortest_robot1_moves(target_code):
        start_pos = numeric_coords['A']
        return find_shortest_sequence(start_pos, target_code, numeric_keypad, get_numeric_coords)

    def find_shortest_robot2_moves(target_robot1_actions):
        start_pos = dir_coords['A']
        return find_shortest_sequence(start_pos, target_robot1_actions, dir_keypad, get_directional_coords)

    def find_shortest_your_moves(target_robot2_actions):
        start_pos = dir_coords['A']
        return find_shortest_sequence(start_pos, target_robot2_actions, dir_keypad, get_directional_coords)

    memo_robot1 = {}
    def shortest_robot1(target):
        if target in memo_robot1:
            return memo_robot1[target]

        q = deque([(numeric_coords['A'], "")])
        visited = {numeric_coords['A']}
        while q:
            current_pos, moves = q.popleft()
            if "".join(moves_made[0] for moves_made in simulate_robot1(moves)) == target:
                memo_robot1[target] = moves
                return moves

            for next_move in ['^', 'v', '<', '>']:
                new_pos = move_robot_arm(numeric_keypad, current_pos, next_move)
                if new_pos and new_pos not in visited:
                    visited.add(new_pos)
                    q.append((new_pos, moves + next_move))
            q.append((current_pos, moves + 'A')) # Press 'A'

        return None

    def simulate_robot1(moves):
        current_pos = numeric_coords['A']
        pressed_buttons = []
        for move in moves:
            if move == 'A':
                if current_pos in numeric_keypad:
                    pressed_buttons.append(numeric_keypad[current_pos])
            else:
                current_pos = move_robot_arm(numeric_keypad, current_pos, move)
                if not current_pos: return None # Moved to gap
        return pressed_buttons

    def move_robot_arm(keypad, current_pos, move):
        row, col = current_pos
        if move == '^': new_pos = (row - 1, col)
        elif move == 'v': new_pos = (row + 1, col)
        elif move == '<': new_pos = (row, col - 1)
        elif move == '>': new_pos = (row, col + 1)
        return new_pos if new_pos in keypad else None

    memo_robot2 = {}
    def shortest_robot2(target_robot1_moves):
        if target_robot1_moves in memo_robot2:
            return memo_robot2[target_robot1_moves]

        q = deque([(dir_coords['A'], "")])
        visited = {dir_coords['A']}
        while q:
            current_pos, moves = q.popleft()
            simulated_robot1_moves = simulate_robot2(moves)
            if simulated_robot1_moves == target_robot1_moves:
                memo_robot2[target_robot1_moves] = moves
                return moves

            for next_move in ['^', 'v', '<', '>']:
                new_pos = move_robot_arm(dir_keypad, current_pos, next_move)
                if new_pos and new_pos not in visited:
                    visited.add(new_pos)
                    q.append((new_pos, moves + next_move))
            q.append((current_pos, moves + 'A'))

        return None

    def simulate_robot2(moves):
        current_pos = dir_coords['A']
        robot1_actions = ""
        for move in moves:
            if move == 'A':
                if current_pos in dir_keypad:
                    robot1_actions += dir_keypad[current_pos]
            else:
                current_pos = move_robot_arm(dir_keypad, current_pos, move)
                if not current_pos: return None
        return robot1_actions

    memo_your = {}
    def shortest_your(target_robot2_moves):
        if target_robot2_moves in memo_your:
            return memo_your[target_robot2_moves]

        q = deque([(dir_coords['A'], "")])
        visited = {dir_coords['A']}
        while q:
            current_pos, moves = q.popleft()
            simulated_robot2_moves = simulate_your(moves)
            if simulated_robot2_moves == target_robot2_moves:
                memo_your[target_robot2_moves] = moves
                return moves

            for next_move in ['^', 'v', '<', '>']:
                new_pos = move_robot_arm(dir_keypad, current_pos, next_move)
                if new_pos and new_pos not in visited:
                    visited.add(new_pos)
                    q.append((new_pos, moves + next_move))
            q.append((current_pos, moves + 'A'))
        return None

    def simulate_your(moves):
        current_pos = dir_coords['A']
        robot2_actions = ""
        for move in moves:
            if move == 'A':
                if current_pos in dir_keypad:
                    robot2_actions += dir_keypad[current_pos]
            else:
                current_pos = move_robot_arm(dir_keypad, current_pos, move)
                if not current_pos: return None
        return robot2_actions

    for code_str in codes:
        shortest_robot1_moves_str = shortest_robot1(code_str)
        if not shortest_robot1_moves_str:
            raise Exception(f"Could not find Robot 1 sequence for {code_str}")

        shortest_robot2_moves_str = shortest_robot2(shortest_robot1_moves_str)
        if not shortest_robot2_moves_str:
            raise Exception(f"Could not find Robot 2 sequence for {code_str}")

        shortest_your_moves_str = shortest_your(shortest_robot2_moves_str)
        if not shortest_your_moves_str:
            raise Exception(f"Could not find Your sequence for {code_str}")

        numeric_part = int(code_str.lstrip('0') or '0')
        complexity = len(shortest_your_moves_str) * numeric_part
        total_complexity += complexity

    return total_complexity

codes = ["029A", "980A", "179A", "456A", "379A"]
result = solve_keypad_puzzle(codes)
print(result)

In [42]:
from collections import deque
from enum import Enum
from functools import cache
from typing import Any, TypeAlias

Grid = list[list[str]]
Position: TypeAlias = tuple[int, int]


numeric_keypad: Grid = [
    ["7", "8", "9"],
    ["4", "5", "6"],
    ["1", "2", "3"],
    ["#", "0", "A"],
]
directional_keypad: Grid = [["#", "^", "A"], ["<", "v", ">"]]


class Face(Enum):
    NORTH = (0, "^")
    EAST = (1, ">")
    SOUTH = (2, "v")
    WEST = (3, "<")

    def symbol(self) -> str:
        return self.value[1]

    @classmethod
    def from_delta(cls, dr: int, dc: int) -> "Face":
        match (dr, dc):
            case (0, 1):
                return cls.EAST
            case (1, 0):
                return cls.SOUTH
            case (0, -1):
                return cls.WEST
            case (-1, 0):
                return cls.NORTH
            case _:
                raise ValueError(f"Invalid delta: ({dr}, {dc})")


def find_paths(
    grid: Grid,
    start_row: int,
    start_col: int,
    end_row: int,
    end_col: int,
):
    queue = deque([(start_row, start_col, Face.NORTH, [])])
    paths = []
    min_length = float("inf")

    while queue:
        r, c, face, path = queue.popleft()

        if len(path) > min_length:
            continue

        if (r, c) == (end_row, end_col):
            if len(path) <= min_length:
                min_length = len(path)
                paths.append("".join(p[2] for p in path) + "A")

            continue

        for dr, dc in ((0, 1), (1, 0), (0, -1), (-1, 0)):
            new_r, new_c = r + dr, c + dc
            if (
                0 <= new_r < len(grid)
                and 0 <= new_c < len(grid[0])
                and grid[new_r][new_c] != "#"
            ):
                face = Face.from_delta(dr, dc)

                new_path = path + [(new_r, new_c, face.symbol())]
                queue.append((new_r, new_c, face, new_path))

    return paths


@cache
def directional_path_length(sequence: str, depth: int) -> int:
    if depth == 0:
        return len(sequence)

    start_row, start_col = find_point(directional_keypad, "A")
    total = 0

    for char in sequence:
        end_row, end_col = find_point(directional_keypad, char)

        paths = find_paths(
            directional_keypad,
            start_row,
            start_col,
            end_row,
            end_col,
        )
        total += min(directional_path_length(path, depth - 1) for path in paths)

        start_row, start_col = end_row, end_col

    return total


def sequence_length(sequence: str, depth: int) -> int:
    start_row, start_col = find_point(numeric_keypad, "A")
    total = 0

    for char in sequence:
        end_row, end_col = find_point(numeric_keypad, char)

        paths = find_paths(numeric_keypad, start_row, start_col, end_row, end_col)
        total += min(directional_path_length(path, depth) for path in paths)

        start_row, start_col = end_row, end_col

    return total


def find_point(grid: Grid, type: str) -> Position:
    return next(find_points(grid, type))


def find_points(grid: Grid, type: str):
    for row in range(len(grid)):
        for col in range(len(grid[0])):
            if grid[row][col] == type:
                yield row, col


def part_1() -> Any:
    codes = [line for line in open("Input/InputDay21P1.txt").read().splitlines()]
    result = 0

    for code in codes:
        id = int("".join([char for char in code if char.isdigit()]))
        length = sequence_length(code, 2)

        result += id * length

    return result


def part_2() -> Any:
    codes = [line for line in open("Input/InputDay21P1.txt").read().splitlines()]
    result = 0

    for code in codes:
        id = int("".join([char for char in code if char.isdigit()]))
        length = sequence_length(code, 25)

        result += id * length

    return result

print(part_2())

223902935165512
